In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from scipy.optimize import minimize, differential_evolution
import os
import json
from susy_qm import calculate_Hamiltonian

In [231]:
potential = 'DW'
cutoff = 8
shots = None

In [232]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [233]:
num_qubits

4

In [234]:
min_eigenvalue

np.complex128(0.8845804438663997+0j)

In [235]:
HP = qml.pauli_decompose(H)

In [261]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
#basis_state = [1,1,1,0] #AHO8
basis_state = [0,1,1,1]

@qml.qnode(dev)
def circuit(times, operator_ham, op_list, trotter_steps=3):

    #qml.BasisState([1]*num_qubits, wires=range(num_qubits)) # + [0]*(num_qubits-1)
    qml.BasisState(basis_state, wires=range(num_qubits))

    for i, (op, _, _) in enumerate(op_list):
        qml.ApproxTimeEvolution(op, time=times[i], n=trotter_steps)

    qml.ApproxTimeEvolution(operator_ham, time=times[-1], n=trotter_steps)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [262]:
def cost_function(times, operator, op_list):

    #t = pnp.tensor(t, requires_grad=True)
    times = pnp.tensor(times, requires_grad=True)
    #grad_fn = qml.grad(circuit)
    #grad = grad_fn(t, operator, op_list)[0]

    #params = pnp.tensor(params, requires_grad=True)
    energy = circuit(times, operator, op_list)
    
    return energy#-abs(grad)

In [263]:
operator_pool = HP
for i in range(num_qubits):
    operator_pool += qml.PauliX(i)
    operator_pool += qml.PauliY(i)
    operator_pool += qml.PauliZ(i)


In [264]:
num_steps = 5
op_list = []

#variables
max_iter = 250
strategy = "randtobest1bin"
tol = 1e-2
atol = 1e-3
popsize = 15

bounds = [(0.001, 2 * np.pi)]

for i in range(num_steps):

    print(f"step: {i}")

    gradients = []
    terms = []
    times = []

    x0 = [0.1] * (len(op_list) + 1)
    bounds = [(0.001, 2 * np.pi) for _ in range(len(op_list) + 1)]

    for op in operator_pool.terms()[1]:

        '''
        res = minimize(
                cost_function,
                x0=x0,
                bounds = [(0.001, 2*np.pi)],
                args=(op, op_list),
                method= "COBYLA",
                options= {'maxiter':10000, 'tol': 1e-8}
            )
        '''
        #'''
        res = differential_evolution(cost_function,
                                    bounds,
                                    args=(op, op_list),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=atol,
                                    strategy=strategy,
                                    popsize=popsize
                                    )
        #'''
        gradients.append(res.fun)
        times.append(res.x)

    if i!=0:
        pre_min_grad = min_grad
    min_arg = np.argmin(gradients)
    min_grad = gradients[min_arg]

    print(f"Min grad: {min_grad}")

    if i!=0:
        if abs(pre_min_grad - min_grad) < 1e-8:
            print("gradient converged")
            break

    min_op = operator_pool[min_arg]
    min_t = times[min_arg]
    print(min_op, min_t)

    op_list.append((min_op,min_t,min_grad))

step: 0
Min grad: 2.374999999999994
3.159526931034373 * (I(0) @ X(1) @ X(2) @ X(3)) [1.57079633]
step: 1
Min grad: 1.093911086754477
1.0 * Y(3) [2.48580936 2.66393433]
step: 2
Min grad: 1.0786618070319391
1.0 * Y(0) [3.48013311 2.86914931 4.71238866]
step: 3
Min grad: 0.9241175735078525
1.0 * Y(2) [0.49716187 6.08913721 1.57079661 2.98392328]
step: 4
Min grad: 0.8951448522868215
1.0 * Y(1) [0.49716188 6.10799988 4.71238805 2.94044275 3.19498255]


In [265]:
hamiltonian_dict = {op: coeff for op, coeff in zip(HP.terms()[1], HP.terms()[0])}
op_coeffs = []
for op, t, grad in op_list:
    coeff = hamiltonian_dict.get(op, None)
    op_coeffs.append((op, coeff, t))

In [266]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def final_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))

    for i, (op, coeff, t) in enumerate(op_coeffs):
        pauli_string = qml.pauli.pauli_word_to_string(op)
        qml.PauliRot(params[i], pauli_string, wires=op.wires)

    return qml.state()

In [267]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_coeffs))
print(qml.draw(final_circuit)(x0))

0: ─╭|Ψ⟩─╭RIXXX(3.25)──RY(1.34)─┤  State
1: ─├|Ψ⟩─├RIXXX(3.25)──RY(2.64)─┤  State
2: ─├|Ψ⟩─├RIXXX(3.25)──RY(1.07)─┤  State
3: ─╰|Ψ⟩─╰RIXXX(3.25)──RY(5.97)─┤  State


In [268]:
def overlap_function(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = final_circuit(params)
    
    overlap = np.vdot(min_eigenvector, ansatz_state)
    cost = np.abs(overlap)**2  

    return (1 - cost)

In [269]:
def hellinger(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = final_circuit(params)
    
    min_eigenvector_prob = np.abs(min_eigenvector)**2
    ansatz_prob = np.abs(ansatz_state)**2
    cost = np.sum(np.sqrt(np.vdot(min_eigenvector_prob, ansatz_prob)))

    return (1 - cost)

In [270]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_coeffs))
o_iters = 10000
o_tol=1e-8

print("Running for overlap")

overlap_res = minimize(
    overlap_function,
    x0,
    method= "COBYLA",
    options= {'maxiter':o_iters, 'tol': o_tol}
)

hf_res = minimize(
    hellinger,
    x0,
    method= "COBYLA",
    options= {'maxiter':o_iters, 'tol': o_tol}
)

Running for overlap


In [271]:
overlap_res.fun

np.float64(0.02856289256240696)

In [272]:
hf_res.fun

np.float64(0.21190712963092517)

In [254]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "DE",
        "num steps": num_steps,
        "basis_state": basis_state,
        "op_list": [str(o[0].terms()[1]) for o in op_list],
        "overlap": overlap_res.fun,
        "hellinger": hf_res.fun
        }

In [255]:
data

{'potential': 'DW',
 'cutoff': 8,
 'optimizer': 'DE',
 'num steps': 5,
 'basis_state': [1, 0, 0, 0],
 'op_list': ['[Y(2)]', '[Y(3)]', '[Y(1)]'],
 'overlap': np.float64(1.0),
 'hellinger': np.float64(1.0)}

In [256]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\TimeEv\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [36]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def test(params):

    qml.BasisState([1,1,1,0], wires=range(num_qubits))

    qml.PauliRot(params[0], 'IXXI', wires=[0,1,2,3])
    qml.PauliRot(params[1], 'Y', wires=[2])
    qml.PauliRot(params[2], 'IIZI', wires=[0,1,2,3])
    qml.PauliRot(params[3], 'IXXZ', wires=[0,1,2,3])

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [ ]:
x0 = np.random.uniform(0, 2 * np.pi, size=4)
print(qml.draw(test)(x0))